In [1]:
# %load notebook_setup.py
#Use %load notebook_setup.py to load these commands at the top of your
#ipython notebook.  Includes running the std_libraries.py file. 

%run std_libraries.py
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
%matplotlib inline
#%matplotlib notebook

what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = otherb
home directory is /home/heineike/
y1000plus_tools dir is /home/heineike/github/y1000plus_tools/
y1000plus data dir is /home/heineike/genomes/y1000plus/
Added /home/heineike/github/y1000plus_tools/ to path
Importing y1000plus_tools.py
home directory is 
y1000plus_tools dir is 
y1000plus data dir is 
Added /home/heineike/github/expression_broad_data/core/ to path
importing io_library.py
Importing io_library.  If autoreload, may need to reset base_dir and data_processing dir 
  io_library.base_dir=base_dir 
 io_library.data_processing_dir = data_processing_dir
sys.path : 

['/home/heineike/github/y1000plus_tools/scripts', '', '/home/lab/envs/seqanalysis/lib/python35.zip', '/home/lab/envs/seqanalysis/lib/python3.5', '/home/lab/envs/seqanalysis/lib/python3.5/plat-linux', '/home/lab/envs/seqanalysis/lib/python3.5/lib-dynload', '/home/lab/envs/seqanalysis/lib/python3.5/site-packages', '/home/lab/envs/seq

In [2]:
#Select species of interest from the y1000plus_species_table

y1000_species = pd.read_csv(y1000plus_dir + "y1000_species_table.csv", index_col=0)

#Removes outgroup species which don't have genomes in the dataset
y1000_species_subset = y1000_species[y1000_species['Major clade']!='outgroup']

# #Select only clades of interest: 
# clades_of_interest = {'Saccharomycetaceae','Saccharomycodaceae','Phaffomycetaceae','CUG-Ser2','CUG-Ser1','Pichiaceae','CUG-Ala'}
# #clades_of_interest = {'Saccharomycetaceae'}
# y1000_species_subset = y1000_species[y1000_species['Major clade'].isin(clades_of_interest)]


In [67]:
gois
y1000_id_list

['110_1144', '110_2005', '110_3023', '110_903', '110_5778']

In [68]:
goi_og_lookup

{'110_1144': 'OG2226',
 '110_2005': 'OG2926',
 '110_3023': 'OG4651',
 '110_5778': 'OG5100',
 '110_903': 'OG4685'}

In [3]:
#For a given list of genes of interest, makes lookup tables for their orthogroups

#Gois should be given by S.Cer systematic name
gois_common_names = ['GPA1', 'STE4', 'STE18', 'FAR1', 'RME1']
gois = io_library.SC_orf_lookup_by_name(gois_common_names)

#Make promoter files for various genes of interest
#Load S.Cer lookup table: 
scer_lookup_fname = y1000plus_dir + os.path.normpath('id_lookups/saccharomyces_cerevisiae.csv')

scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)
# target_y1000_id = scer_lookup.loc[goi,'y1000_id']

y1000_ids = dict(scer_lookup.loc[gois,'y1000_id'])
y1000_id_list = [y1000_ids[goi] for goi in gois]

goi_og_lookup, og_genes_lookup = y1000plus_tools.make_og_genes_lookup(y1000_id_list, y1000_species_subset)

In [11]:
#Extract promoters for all gois
L_prom = 1000

for jj, goi_common_name in enumerate(gois_common_names):
    print(goi_common_name)
    goi = gois[jj]
    goi_y1000_id = scer_lookup.loc[goi]['y1000_id']
    og = goi_og_lookup[goi_y1000_id]
    og_genes = og_genes_lookup[og]

    #For a given orthogroup extract promoters
    og_genes_y1000_gene_id = y1000plus_tools.extract_promoters(L_prom, og, og_genes, y1000_species_subset, fname_string=goi_common_name)
    print(goi_common_name + ' promoter extraction complete')

GPA1
promoter region extends past the scaffold, genome_name = yHMPu5000034673_blastobotrys_nivea_160519 Gene = Seq_6855, L_prom = 1000
promoter region extends past the scaffold, genome_name = yHMPu5000034865_zygosaccharomyces_kombuchaensis_160519 Gene = Seq_2802, L_prom = 1000
promoter region extends past the scaffold, genome_name = yHMPu5000035696_hanseniaspora_singularis_160519 Gene = Seq_4092, L_prom = 1000
12_469
promoter region extends past the scaffold, genome_name = yHMPu5000034952_citeromyces_siamensis_160519 Gene = Seq_5799, L_prom = 1000
S.Cerevisiae promoter is only 700 bases long, but L_prom=1000
GPA1 promoter extraction complete
STE4
promoter region extends past the scaffold, genome_name = yHMPu5000034673_blastobotrys_nivea_160519 Gene = Seq_1241, L_prom = 1000
12_3217
S.Cerevisiae promoter is only 700 bases long, but L_prom=1000
STE4 promoter extraction complete
STE18
promoter region extends past the scaffold, genome_name = yHMPu5000034673_blastobotrys_nivea_160519 Gene =

# For a given gene, Make a multiple sequence alignment

In [4]:
goi_common_name = 'RME1' #already did GPA1

print(goi_common_name)
goi = dict(zip(gois_common_names, gois))[goi_common_name]
goi_y1000_id = scer_lookup.loc[goi]['y1000_id']
og = goi_og_lookup[goi_y1000_id]
og_genes = og_genes_lookup[og]

RME1


In [8]:
#choose outgroup species (important for making protein trees from multiple sequence alignments) 
outgroup = 'Lipomyces japonicus' # 'Tortispora starmeri' #'Lipomyces starkeyi'   #Lipomyces starkeyi is a good outgroup for all the species
# GPA1: Lipomyces starkeyi
# RME1: Lipomyces japonicus
outgroup_orig_genome = y1000_species[y1000_species['Species name']==outgroup]['original_genome_id'].values[0]

#Test to see if orthogroup has something from outgroup: 

y1000_id_outgroup = y1000_species_subset[y1000_species_subset['Species name']==outgroup]['spec_og_id'].values[0]

outgroup_gene = ''
for gene in og_genes: 
    if gene.split('_')[0]== str(y1000_id_outgroup):
        outgroup_gene = gene

if outgroup_gene =='': 
    print(outgroup + ' has no proteins in og ' + og + ' for ' + goi_common_name + '. Pick another outgroup')
else: 
    print('outgroup gene = ' + outgroup_gene)
    #Results: 
#GPM2/GPM3 no ortholog in Y. Lipolitica
#TPK1/TPK3 no ortholog in Y. lipolitica (interesting...)  
# Need to find new outgroup for those



outgroup gene = 214_6372


In [6]:
#Check to see which major clades / species are present in the orthogroup if 
#there is no good outgroup
specs_in_og = []
for gene in og_genes: 
    specs_in_og.append(gene.split('_')[0])

set(y1000_species_subset[y1000_species_subset['spec_og_id'].isin(set(specs_in_og))]['Major clade'])

{'Alloascoideaceae',
 'CUG-Ala',
 'CUG-Ser1',
 'CUG-Ser2',
 'Dipodascaceae/Trichomonascaceae',
 'Lipomycetaceae',
 'Phaffomycetaceae',
 'Pichiaceae',
 'Saccharomycetaceae',
 'Saccharomycodaceae',
 'Trigonopsidaceae'}

In [7]:
#Lipomyces is a good outgroup set: 
# 'Lipomyces oligophaga', 'Lipomyces starkeyi',
# 'Lipomyces suomiensis', 'Lipomyces doorenjongii',
# 'Lipomyces mesembrius', 'Lipomyces kononenkoae',
# 'Lipomyces japonicus', 'Lipomyces lipofer', 'Lipomyces arxii'

#'Lipomycetaceae'

#after that try: 
#Trigonopsidaceae
#Dipodascaceae/Trichomonascaceae
#'Alloascoideaceae'
#'Sporopachydermia clade'
#'Pichiaceae'
#'CUG-Ser2'
#'Phaffomycetaceae'
# 'Saccharomycodaceae'

#Test to see if orthogroup has anything from an outgroup clade: 
outgroup_clade =  'Lipomycetaceae'
y1000_id_possible_outgroups = y1000_species_subset[y1000_species_subset['Major clade']==outgroup_clade]['spec_og_id'].values
y1000_id_possible_outgroups = [str(y1000_spec_id) for y1000_spec_id in y1000_id_possible_outgroups]


outgroup_genes = []
for gene in og_genes: 
    if gene.split('_')[0] in y1000_id_possible_outgroups:
        outgroup_genes.append(gene)

if len(outgroup_genes) ==0: 
    print(goi_common_name + ' orthogroup ' + og + ' has no proteins in the major clade ' + outgroup_clade + '. Pick another outgroup')
else:
    print('potential outgroup genes: ')
    print(outgroup_genes)
    outgroup_spec_ids = [outgroup_gene.split('_')[0] for outgroup_gene in outgroup_genes]
    print('potential outgroup species names: ')
    print(y1000_species_subset[y1000_species_subset['spec_og_id'].isin(outgroup_spec_ids)]['Species name'])

potential outgroup genes: 
['214_6372']
potential outgroup species names: 
ID
202    Lipomyces japonicus
Name: Species name, dtype: object


In [9]:
#extract protein sequence: 
protein_fname = goi_common_name + '_' + og
y1000plus_tools.extract_protein_seqs(og_genes, protein_fname, y1000_species_subset)
print(goi_common_name + ' protein sequence extraction complete')

yHMPu5000035645_yarrowia_divulgata_160519
yHMPu5000035689_candida_mycetangii_160613
lachancea_kluyveri
yHMPu5000034967_candida_freyschussii_160519
yHMPu5000035629_yueomyces_sinensis_160519
yHMPu5000035639_wickerhamomyces_canadensis_160613
yHMPu5000035690_candida_vartiovaarae_160613
yHAB159_kazachstania_solicola_160519
yHMPu5000035673_candida_orba_160613
yHMPu5000035658_starmera_amethionina_160613
yHMPu5000035667_kurtzmaniella_cleridarum_160928
yHAB158_kazachstania_viticola_160519
zygosaccharomyces_bailii
yHMPu5000035037_candida_montana_160519
nakaseomyces_nivariensis
yHMPu5000035696_hanseniaspora_singularis_160519
yHAB164_kazachstania_aerobia_160519
yHMPu5000034675_blastobotrys_mokoenaii_160519
yHMPu5000034674_blastobotrys_muscicola_160519
kluyveromyces_marxianus
yHMPu5000034667_blastobotrys_serpentis_160519
nakazawaea_peltata
yHAB134_kazachstania_turicensis_160519
yHMPu5000035687_cyberlindnera_suaveolens_160613
yHMPu5000034758_lipomyces_japonicus_160519
yHMPu5000034939_komagataella_ps

In [17]:
#Mafft command for MSA and IQtree command for phylogenetic tree

# Mafft command 

#Make tree: 

#Use Mafft to construct an MSA

#goi_common_name and og already set above

protein_fname_base = y1000plus_dir + os.path.normpath('proteins_og/' + goi_common_name + '_' + og)


#Command: 
mafft_command = ['mafft', '--genafpair', '--maxiterate', '1000', 
                protein_fname_base + '.fasta', 
                '>',
                protein_fname_base + '_aln.fasta']

#Running it from this notebook seems slow
#!{" ".join(mafft_command)}

print(" ".join(mafft_command))

#This is the E-ENS-i algorithm which is suitable for conserved regions separated by long unaligned gaps. https://mafft.cbrc.jp/alignment/software/algorithms/algorithms.html
#tree command: (designated outgroup)

#get outgroup gene's name

if outgroup_gene == None: 
    print('No outgroup gene designated - need to pick outgroup gene')


protein_fname_aln = protein_fname_base + '_aln.fasta'
prot_seqs = SeqIO.parse(protein_fname_aln, "fasta")  

for seq in prot_seqs: 
    y1000_id = seq.description.split(' ')[1].split('=')[1]
    if y1000_id == outgroup_gene:
        outgroup_id = seq.id

trimal_command = ['/home/lab/trimal/source/trimal',
                  '-in', protein_fname_aln, 
                  '-out', protein_fname_base + '_aln_trimmed.fasta', 
                  '-gappyout']

print(" ".join(trimal_command))

#
# outgroup_gene = None
# prot_seqs = SeqIO.parse(protein_fname_base + '.fasta', "fasta")
# for seq in prot_seqs: 
#     if seq.id.split('Seq')[0]== outgroup_orig_genome+'_':
#     #if seq.id.split('Seq')[0]=='hanseniaspora_vinae_':
#         outgroup_gene = seq.id
        

print("Note: for iqtree to run you need to activate the seqanalysis python environment: . activate /home/lab/envs/seqanalysis")

iqtree_command = ["iqtree", 
                  "-s" , protein_fname_base + '_aln_trimmed.fasta',
                  "-nt", "AUTO",  #automatically determines number of threads 
                  "-o", outgroup_id]
print(" ".join(iqtree_command))

#old trimming protocol - open up on Jalview and trim by eye. Save as _trimmed.fasta, then find and replace
# all "/" at the end of each gene name with a space. 


# #Use the MSA to build a tree

# #from that tree for all post WGD species, 
#     #see if gene is closer to "low" or "high" goi.  Assign pre_wgh, low, or high label
#     #could verify that outgroup doesn't appear between target and S.Cer low or high gene as well. 
    
# #Order search results based on the tree


mafft --genafpair --maxiterate 1000 /home/heineike/genomes/y1000plus/proteins_og/RME1_OG5100.fasta > /home/heineike/genomes/y1000plus/proteins_og/RME1_OG5100_aln.fasta
/home/lab/trimal/source/trimal -in /home/heineike/genomes/y1000plus/proteins_og/RME1_OG5100_aln.fasta -out /home/heineike/genomes/y1000plus/proteins_og/RME1_OG5100_aln_trimmed.fasta -gappyout
Note: for iqtree to run you need to activate the seqanalysis python environment: . activate /home/lab/envs/seqanalysis
iqtree -s /home/heineike/genomes/y1000plus/proteins_og/RME1_OG5100_aln_trimmed.fasta -nt AUTO -o yHMPu5000034758_lipomyces_japonicus_160519_Seq_6373


In [111]:
#Convert promoter fasta for FIMO, also make dictionary for converting back

fasta_in = '/home/heineike/genomes/y1000plus/promoter_sets/' + og + '_' + goi_common_name + '.fasta'
fasta_out = '/home/heineike/genomes/y1000plus/promoter_sets/' + og +'_' + goi_common_name + '_fimo.fasta'

gene_id_to_y1000_id = {}
with open(fasta_out,'w') as f_out: 
    with open(fasta_in,'r') as f_in: 
        for line in f_in: 
            line_out = line
            if line[0]=='>':
                y1000_id = line.split()[1].split('=')[1]
                line_out = '>' + y1000_id + '\n'
                species = line.split()[0].split('=')[1]
                gene_id = line.split()[2].split('=')[1]
                gene_id_to_y1000_id[species + '@' + gene_id] = y1000_id
            f_out.write(line_out)

            

In [88]:
#Run FIMO to get hit results
        
#Use Fimo to search for promoter binding sites: 

#(motif_name: (thresh, motif_fname, motif_in_file))
motif_hit_params = {'a1alpha2': (1e-3, base_dir + os.path.normpath('data/a1alpha2_RME1_francesca_kyle/A1Alpha2_MEME_output.meme'), 'All'),
                    'RME1':(1e-3, base_dir + os.path.normpath('data/a1alpha2_RME1_francesca_kyle/RME1_MA0370.1.meme'), 'All'),
                    'alpha2': (1e-3, base_dir + os.path.normpath('data/a1alpha2_RME1_francesca_kyle/ALPHA2_MA0328.1.meme'), 'All')
                   }

fimo_hits = {}
promoters_prefix = og + '_' + goi_common_name
for motif_name, (thresh, motif_fname, motif_in_file) in motif_hit_params.items(): 
    fimo_hits_motif = y1000plus_tools.promoter_scan_fimo(promoters_prefix, motif_name, motif_fname, thresh)
    fimo_hits[motif_name] = fimo_hits_motif

# promoter_fname = y1000plus_dir + 'promoter_sets/' + og + '_' + goi_common_name + '_fimo.fasta'
# output_dir = y1000plus_dir + 'fimo_results' + os.sep
# motif_in_file = 'All'  #If there are multiple motifs in a file, this picks them out. 

# if motif_in_file == "All":
#     motif_arg = []
# else:
#     motif_arg = ["--motif",motif_in_file]


# for motif_name, (thresh, motif_fname) in motif_hit_params.items(): 
    
#     fname_prefix = og + '_' + goi_common_name + '_' + motif_name
#     fimo_command = ([ home_dir + "meme/bin/fimo",
#                       "--oc", output_dir,
#                       "--verbosity", "1",
#                       "--thresh", str(thresh)] +
#                      motif_arg + 
#                      [ motif_fname,
#                        promoter_fname]
#                    )
    
#     print('fimo command:\n' + ' '.join(fimo_command))

#     fimo_output = subprocess.run(fimo_command,stdout = subprocess.PIPE) 

#     print("fimo output return code = " + str(fimo_output.returncode))

#     #change file prefix and delete output other than .txt file
#     files_to_change = ['cisml.xml', 'fimo.html', 'fimo.tsv','fimo.xml', 'fimo.gff']

#     for file_to_change in files_to_change: 
#         full_file_to_change = output_dir +  file_to_change   
#         fimo_fname_out = output_dir + fname_prefix + '_' + file_to_change
#         os.rename(full_file_to_change, fimo_fname_out)
#         #os.remove(full_file_to_change)

alpha2
fimo command:
/home/heineike/meme/bin/fimo --oc /home/heineike/genomes/y1000plus/fimo_results/ --verbosity 1 --thresh 0.001 /home/heineike/github/y1000plus_tools/data/a1alpha2_RME1_francesca_kyle/ALPHA2_MA0328.1.meme /home/heineike/genomes/y1000plus/promoter_sets/OG2226_GPA1_fimo.fasta
fimo output return code = 0
a1alpha2
fimo command:
/home/heineike/meme/bin/fimo --oc /home/heineike/genomes/y1000plus/fimo_results/ --verbosity 1 --thresh 0.001 /home/heineike/github/y1000plus_tools/data/a1alpha2_RME1_francesca_kyle/A1Alpha2_MEME_output.meme /home/heineike/genomes/y1000plus/promoter_sets/OG2226_GPA1_fimo.fasta
fimo output return code = 0
RME1
fimo command:
/home/heineike/meme/bin/fimo --oc /home/heineike/genomes/y1000plus/fimo_results/ --verbosity 1 --thresh 0.001 /home/heineike/github/y1000plus_tools/data/a1alpha2_RME1_francesca_kyle/RME1_MA0370.1.meme /home/heineike/genomes/y1000plus/promoter_sets/OG2226_GPA1_fimo.fasta
fimo output return code = 0


297_1734
320_2133
58_1069
249_2404
291_188
293_3911
321_884
156_2982
156_3768
156_4799
310_4072
302_5054
306_5171
155_3300
341_3557
265_4577
98_4768
324_2442
158_10
205_3687
204_703
51_2989
199_2664
99_1960
99_4923
147_2452
318_3245
214_6372
242_4182
271_1958
279_1699
101_4916
14_1198
146_2143
65_571
159_4479
144_4715
277_76
50_2982
326_4441
142_5169
230_5115
232_1048
280_2480
223_204
35_2167
61_2483
92_4667
258_5098
234_3359
275_2014
275_5639
231_1017
208_952
298_2806
342_305
44_2341
300_3364
9_1149
228_295
237_1571
308_1688
308_471
13_3904
269_2329
226_1802
207_1284
167_4550
136_2312
56_4151
238_57
270_1720
229_5550
224_5610
317_1682
323_690
6_3253
6_1902
110_5778
100_95
164_4602
305_3138
322_5805
149_5140
143_725
5_5208
154_1052
106_2303
106_2619
97_1645
239_1062
239_2725
191_3035
95_213
135_4083
148_3840
171_5993
114_161
109_4006
139_744
112_2346
140_6405
233_1368
233_2811
111_2056
46_3847
289_4328
206_4677
307_5805
283_4462
268_4265
257_478
250_4622
59_4621
52_3093
189_5349
105_45

In [118]:
## add motifs to the same promoter image 

#Visualize the tree with promoter binding sites localized 

goi_common_name =  'GPA1'
goi = dict(zip(gois_common_names, gois))[goi_common_name]
goi_y1000_id = scer_lookup.loc[goi]['y1000_id']
og = goi_og_lookup[goi_y1000_id]
#og_genes = og_genes_lookup[og]

sacc_families = {'Candida': 'Post_WGH',
                 'Kazachstania': 'Post_WGH',
                 'Nakaseomyces': 'Post_WGH',
                 'Naumovozyma': 'Post_WGH',
                 'Saccharomyces': 'Post_WGH',
                 'Tetrapisispora': 'Post_WGH',
                 'Vanderwaltozyma': 'Post_WGH',
                 'Yueomyces': 'Post_WGH',
                 'Zygosaccharomyces': 'ZT',
                 'Zygotorulaspora': 'ZT',
                 'Torulaspora': 'ZT',
                 'Kluyveromyces': 'KLE',
                 'Lachancea': 'KLE',
                 'Eremothecium': 'KLE',
                 'Ashbya': 'KLE'
                }


#less_nonsacc=False    #flag for filtering some species

#Color Node by species: 
# sacc_colors = {'KLE': "#C6AFE9", 
#                'ZT': "YellowGreen",
#                'Post_WGH': "LightYellow"
#               }

# maj_clade_colors = {'Saccharomycodaceae': "BlanchedAlmond",
#     'Phaffomycetaceae': "Bisque",
#     'CUG-Ser2': 'NavajoWhite',
#     'CUG-Ser1': "Wheat",
#     'Pichiaceae': "BurlyWood",
#     'CUG-Ala':"Tan"
#     }

#pre_wgd_purple = "#bb95d6"

outgroup_color = 'lightgray'
between_color = 'plum'
maj_clade_colors = {'Saccharomycetaceae': "LightYellow",
    'Saccharomycodaceae': between_color,
    'Phaffomycetaceae': between_color,
    'CUG-Ser2': between_color,
    'CUG-Ser1': "YellowGreen",
    'Pichiaceae': outgroup_color,
    'CUG-Ala': outgroup_color,
    'Lipomycetaceae': outgroup_color, 
    'Trigonopsidaceae': outgroup_color, 
    'Dipodascaceae/Trichomonascaceae': 'darkgray',   #outgroup
    'Alloascoideaceae': outgroup_color, 
    'Sporopachydermia clade': outgroup_color
    }


protein_fname_base = y1000plus_dir + os.path.normpath('proteins_og/' + goi_common_name + '_' + og)
tree_fname = protein_fname_base + '_aln_trimmed.fasta.treefile'
t = Tree(tree_fname)
t.ladderize()
L_prom = 1000
height = 15
seq = '-'*L_prom


cmap_base = {'a1alpha2': 'Reds', 'RME1': 'Blues'}
cmap_low_cutoff = 0.2

cmaps = {}
for motif_name, cmap_base_name in cmap_base.items():
    cmap = cm.get_cmap(cmap_base_name)
    cmap_trunc = colors.LinearSegmentedColormap.from_list(cmap_base_name + '_' + 'trunc', cmap(np.linspace(cmap_low_cutoff,1.0,100)))
    cmaps[motif_name] = cmap_trunc

#(length, thresh)
motif_params = {'a1alpha2': (16,-np.log10(1.0e-3)), 'RME1': (16,-np.log10(1.0e-03))}


found_motifs_dict = {}
for node in t.iter_leaves():  
    found_motifs_dict[node.name]=[]

    
cmap_norm_score = {}
for motif_name, (motif_len, motif_score_thresh) in motif_params.items(): 
    print(motif_name)
    fimo_hits = pd.read_table(y1000plus_dir + 'fimo_results/' + og + '_' + goi_common_name + '_' + motif_name + '_fimo.tsv',
                         engine='python', skipfooter=4)

    #set up colormap
    cmap_score = cmaps[motif_name]
    
    pvals_nlog10 = -np.log10(fimo_hits['p-value'])
    vmin = min(pvals_nlog10)
    vmax = max(pvals_nlog10)
    norm_score = colors.Normalize(vmin=vmin, vmax=vmax)
    
    cmap_norm_score[motif_name] = (cmap_score, norm_score)

    #For each node in the tree:
    for node in t.iter_leaves():  
        if 'candida_albicans' in node.name: 
            species = 'candida_albicans'
            gene_id = species + '@' + node.name.split(species)[1][1:]
        elif 'saccharomyces_cerevisiae' in node.name: 
            species = 'saccharomyces_cerevisiae'
            gene_id = species + '@' + node.name.split(species)[1][1:]
        else: 
            #Get the promoter sequence with motif info, make it into a motif list
            species = '_'.join(node.name.split('_')[:-2])
            #here gene_id includes the species
            gene_id = species + '@' + '_'.join(node.name.split('_')[-2:])

        #color node by major clade / family if in Sacch clade
        row = y1000_species_subset[y1000_species_subset['original_genome_id']==species]
        maj_clade = row['Major clade'].values[0]

#         if maj_clade == 'Saccharomycetaceae':
#             genus = row['Genus'].values[0]
#             node_color = sacc_colors[sacc_families[genus]]
#         elif species == 'yarrowia_lipolytica':
#         #elif species == 'hanseniaspora_vinae':
#             node_color = 'LightGrey'
#        else: 
        node_color = maj_clade_colors[maj_clade]

        nstyle = NodeStyle()
        nstyle['bgcolor']=node_color
        node.set_style(nstyle)

        y1000_id = gene_id_to_y1000_id[gene_id] 

        if y1000_id in set(fimo_hits['sequence_name']): 
            hits = fimo_hits[fimo_hits['sequence_name']==y1000_id]

            for hit in hits.iterrows():
                direction = hit[1]['strand']
                shape = '>'
                start = hit[1]['start']
                stop = hit[1]['start'] + motif_len
                if direction == '-':
                    shape = '<'
                    start = hit[1]['start']-motif_len
                    stop = hit[1]['start']
                pval_neglog10 = -np.log10(hit[1]['p-value'])
                if pval_neglog10>motif_score_thresh: 
                    rgb = colors.to_hex(cmap_score(norm_score(pval_neglog10)))
                    found_motifs_dict[node.name].append([start,stop,shape,None, height, "black", rgb, None])



for node in t.iter_leaves():  
    seqFace = SeqMotifFace(seq, motifs=found_motifs_dict[node.name], seq_format="-")
    node.add_face(seqFace, column=0, position="aligned")
#     if gene_id == 'saccharomyces_uvarum@Seq_2805':
#         print(gene_id)
#         a = node
            

#t.render('%%inline')
fname = home_dir + 'scratch/' + goi_common_name + '.svg'
t.render(fname)

a1alpha2
RME1


{'faces': [[126.46596593878014,
   3553.0,
   350.46596593878013,
   3569.0,
   447,
   'wickerhamomyces_ciferrii_Seq_665'],
  [638.2970615063133, 3560.5, 917.2970615063133, 3561.5, 447, ''],
  [916.2970615063133,
   3553.0425410992134,
   933.9988816096014,
   3568.957458900786,
   447,
   ''],
  [933.2970615063133, 3560.5, 940.2970615063133, 3561.5, 447, ''],
  [939.2970615063133,
   3553.0425410992134,
   956.9988816096014,
   3568.957458900786,
   447,
   ''],
  [956.2970615063133, 3560.5, 994.2970615063133, 3561.5, 447, ''],
  [993.2970615063133,
   3553.0425410992134,
   1010.9988816096014,
   3568.957458900786,
   447,
   ''],
  [994.2970615063133,
   3553.0425410992134,
   1011.9988816096014,
   3568.957458900786,
   447,
   ''],
  [1011.2970615063133, 3560.5, 1124.2970615063132, 3561.5, 447, ''],
  [1123.2970615063132,
   3553.0425410992134,
   1140.9988816096013,
   3568.957458900786,
   447,
   ''],
  [1140.2970615063132, 3560.5, 1279.2970615063132, 3561.5, 447, ''],
  [1278

In [115]:
species

'saccharomyces'